In [162]:
import pickle
import pandas as pd
import numpy as np

In [163]:
movies=pd.read_csv('movie.csv')
ratings=pd.read_csv('rating.csv')

In [164]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [165]:
def convert_to_upper(text):
    return text.upper()

In [166]:
def remove_space(text):
    return text.strip()

In [167]:
movies['title']=movies['title'].apply(convert_to_upper)
movies['title']=movies['title'].apply(remove_space)
movies['genres']=movies['genres'].apply(convert_to_upper)

In [168]:
movies

,movieId,title,genres
0,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY
1,2,JUMANJI (1995),ADVENTURE|CHILDREN|FANTASY
2,3,GRUMPIER OLD MEN (1995),COMEDY|ROMANCE
3,4,WAITING TO EXHALE (1995),COMEDY|DRAMA|ROMANCE
4,5,FATHER OF THE BRIDE PART II (1995),COMEDY
...,...,...,...
27273,131254,KEIN BUND FÜR'S LEBEN (2007),COMEDY
27274,131256,"FEUER, EIS & DOSENBIER (2002)",COMEDY
27275,131258,THE PIRATES (2014),ADVENTURE
27276,131260,RENTUN RUUSU (2001),(NO GENRES LISTED)


In [169]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [170]:
ratings.drop(columns='timestamp',inplace=True)

In [171]:
ratings.shape

(20000263, 3)

In [172]:
movies.shape

(27278, 3)

In [173]:
x=ratings['userId'].value_counts()>700
movie_lover=x[x].index.tolist()
ratings=ratings[ratings['userId'].isin(movie_lover)]
ratings.shape

(4535739, 3)

In [174]:
df=pd.merge(movies,ratings,on='movieId')

In [175]:
df.reset_index(drop=True,inplace=True)

In [176]:
df

,movieId,title,genres,userId,rating
0,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,54,4.0
1,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,116,3.0
2,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,156,5.0
3,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,208,4.0
4,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,298,4.0
...,...,...,...,...,...
4535734,131252,FORKLIFT DRIVER KLAUS: THE FIRST DAY ON THE JO...,COMEDY|HORROR,79570,4.0
4535735,131254,KEIN BUND FÜR'S LEBEN (2007),COMEDY,79570,4.0
4535736,131256,"FEUER, EIS & DOSENBIER (2002)",COMEDY,79570,4.0
4535737,131258,THE PIRATES (2014),ADVENTURE,28906,2.5


In [177]:
df.shape

(4535739, 5)

In [178]:
df['title'].nunique()

25639

In [179]:
df_copy=df.copy()

In [180]:
movie_ratingCount = (df_copy.groupby(by = ['title'])['rating'].count().reset_index().
                     rename(columns = {'rating': 'totalRatingCount'})[['title', 'totalRatingCount']])
movie_ratingCount.head()

,title,totalRatingCount
0,#CHICAGOGIRL: THE SOCIAL NETWORK TAKES ON A DI...,2
1,$ (DOLLARS) (1971),7
2,$5 A DAY (2008),9
3,$9.99 (2008),15
4,$ELLEBRITY (SELLEBRITY) (2012),1


In [181]:
movie_ratingCount.nunique()

title               25639
totalRatingCount     1875
dtype: int64

In [182]:
rating_with_totalRatingCount = df_copy.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,movieId,title,genres,userId,rating,totalRatingCount
0,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,54,4.0,3510
1,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,116,3.0,3510
2,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,156,5.0,3510
3,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,208,4.0,3510
4,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,298,4.0,3510


In [183]:
rating_with_totalRatingCount.shape

(4535739, 6)

In [184]:
rating_with_totalRatingCount.nunique()

movieId             25652
title               25639
genres               1309
userId               3989
rating                 10
totalRatingCount     1875
dtype: int64

In [185]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,movieId,title,genres,userId,rating,totalRatingCount
0,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,54,4.0,3510
1,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,116,3.0,3510
2,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,156,5.0,3510
3,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,208,4.0,3510
4,1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,298,4.0,3510


In [186]:
rating_popular_movie.shape

(4376281, 6)

In [187]:
rating_popular_movie.nunique()

movieId             8877
title               8870
genres               878
userId              3989
rating                10
totalRatingCount    1826
dtype: int64

In [188]:
## First lets create a Pivot matrix

movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,54,104,116,156,208,298,347,348,359,367,...,138208,138254,138270,138301,138307,138325,138382,138397,138406,138437
title,,,,,,,,,,,,,,,,,,,,,
"'BURBS, THE (1989)",3.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
'NIGHT MOTHER (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'ROUND MIDNIGHT (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'TIL THERE WAS YOU (1997),0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) DAYS OF SUMMER (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,...,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.5


In [189]:
movie_names=movie_features_df.index.tolist()

In [190]:
movie_names

["'BURBS, THE (1989)",
 "'NIGHT MOTHER (1986)",
 "'ROUND MIDNIGHT (1986)",
 "'TIL THERE WAS YOU (1997)",
 '(500) DAYS OF SUMMER (2009)',
 '*BATTERIES NOT INCLUDED (1987)',
 '...AND JUSTICE FOR ALL (1979)',
 '10 (1979)',
 '10 ITEMS OR LESS (2006)',
 '10 RILLINGTON PLACE (1971)',
 '10 THINGS I HATE ABOUT YOU (1999)',
 '10 TO MIDNIGHT (1983)',
 '10,000 BC (2008)',
 '100 GIRLS (2000)',
 '101 DALMATIANS (1996)',
 '101 DALMATIANS (ONE HUNDRED AND ONE DALMATIANS) (1961)',
 '101 REYKJAVIK (101 REYKJAVÍK) (2000)',
 '102 DALMATIANS (2000)',
 '10TH KINGDOM, THE (2000)',
 '11\'09\\\\"01 - SEPTEMBER 11 (2002)"',
 '11:14 (2003)',
 '12 ANGRY MEN (1957)',
 '12 ANGRY MEN (1997)',
 '12 ROUNDS (2009)',
 '12 YEARS A SLAVE (2013)',
 '127 HOURS (2010)',
 '13 ASSASSINS (JÛSAN-NIN NO SHIKAKU) (2010)',
 '13 GHOSTS (1960)',
 '13 GOING ON 30 (2004)',
 '13 TZAMETI (2005)',
 '13TH WARRIOR, THE (1999)',
 '1408 (2007)',
 '1492: CONQUEST OF PARADISE (1992)',
 '15 MINUTES (2001)',
 '16 BLOCKS (2006)',
 '17 AGAIN (2009

In [193]:
pickle.dump(movie_names,open('movie_names_remake.pkl','wb'))

In [194]:
pickle.dump(movie_features_df,open('movie_features_df_remake.pkl','wb'))

In [113]:
from scipy.sparse import csr_matrix
movie_features_df_matrix = csr_matrix(movie_features_df.values)


from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [114]:
#pickle.dump(model_knn,open('model_remake.pkl','wb'))